In [25]:
# Import Libraries
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from keras import regularizers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [26]:
# Import Data (Insert your path here)
path_A = '/Users/las/Documents/GitHub/REU/Data/data_A.csv'
path_B = '/Users/las/Documents/GitHub/REU/Data/data_B.csv'
df_A = pd.read_csv(path_A)
df_B = pd.read_csv(path_B)
df_A[df_A.no % 2==1] = df_A[df_A.no % 2==1] + 1 
df_B[df_B.no % 2==1] = df_B[df_B.no % 2==1] + 1 

In [27]:
# Create Training and Validation Data (A)
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(
    df_A.drop("no", axis=1).astype('float64'),
    df_A["no"]//2, 
    test_size=0.2, random_state=42)

X_train_A, X_test_A, y_train_A, y_test_A = np.asarray(X_train_A), np.asarray(X_test_A), np.asarray(y_train_A), np.asarray(y_test_A)
print(f"(A): There a {len(X_train_A)} training samples and {len(X_test_A)} test samples.")

# Create Training and Validation Data (B)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(
    df_B.drop("no", axis=1).astype('float64'),
    df_B["no"]//2, 
    test_size=0.2, random_state=42)

X_train_B, X_test_B, y_train_B, y_test_B = np.asarray(X_train_B), np.asarray(X_test_B), np.asarray(y_train_B), np.asarray(y_test_B)
print(f"(B): There a {len(X_train_B)} training samples and {len(X_test_B)} test samples.")

# Create Training and Validation Data (AB)
X_train_AB, X_test_AB, y_train_AB, y_test_AB = train_test_split(
    df_A.drop("no", axis=1).append(df_B.drop("no", axis=1)).astype('float64'),
    df_A.append(df_B)["no"]//2, 
    test_size=0.2, random_state=42)

X_train_AB, X_test_AB, y_train_AB, y_test_AB = np.asarray(X_train_AB), np.asarray(X_test_AB), np.asarray(y_train_AB), np.asarray(y_test_AB)
print(f"(AB): There a {len(X_train_AB)} training samples and {len(X_test_AB)} test samples.")

(A): There a 40000 training samples and 10000 test samples.
(B): There a 40000 training samples and 10000 test samples.
(AB): There a 80000 training samples and 20000 test samples.


In [28]:
# Set Up Network Architectures
model_A = keras.Sequential([
    keras.layers.Dense(units=1000, activation='relu', input_shape=(18,)),
    keras.layers.Dense(units=1000, activation='relu'),
    keras.layers.Dense(units=93, activation='softmax')
])

model_B = keras.Sequential([
    keras.layers.Dense(units=1000, activation='relu', input_shape=(18,)),
    keras.layers.Dense(units=1000, activation='relu'),
    keras.layers.Dense(units=93, activation='softmax')
])

model_AB = keras.Sequential([
    keras.layers.Dense(units=1000, activation='relu', input_shape=(18,)),
    keras.layers.Dense(units=1000, activation='relu'),
    keras.layers.Dense(units=93, activation='softmax')
])
model_A.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 1000)              19000     
_________________________________________________________________
dense_28 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_29 (Dense)             (None, 93)                93093     
Total params: 1,113,093
Trainable params: 1,113,093
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Set up training
sce = keras.losses.SparseCategoricalCrossentropy()
model_A.compile(optimizer='Adam', loss=sce, metrics=['accuracy', sce])
model_B.compile(optimizer='Adam', loss=sce, metrics=['accuracy', sce])
model_AB.compile(optimizer='Adam', loss=sce, metrics=['accuracy', sce])

In [30]:
# Train models
history_A = model_A.fit(
    X_train_A,
    y_train_A,
    verbose=2,
    epochs=25,
    batch_size=64,
    validation_data=(
        X_test_A,
        y_test_A
    )
)

history_B = model_B.fit(
    X_train_B,
    y_train_B,
    verbose=2,
    epochs=25,
    batch_size=64,
    validation_data=(
        X_test_B,
        y_test_B
    )
)

history_AB = model_AB.fit(
    X_train_AB,
    y_train_AB,
    verbose=2,
    epochs=15,
    batch_size=64,
    validation_data=(
        X_test_AB,
        y_test_AB
    )
)


Train on 40000 samples, validate on 10000 samples
Epoch 1/25


In [ ]:
# Graphs
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].plot(history_A.history['loss'], label="Training Loss") 
axs[0].plot(history_A.history['val_loss'], label="Validation Loss")
axs[0].set_title("Loss_A")
axs[0].legend(loc="upper right")
axs[1].plot(history_A.history['accuracy'], label="Trainig Accuracy") 
axs[1].plot(history_A.history['val_accuracy'], label="Validation Accuracy")
axs[1].legend(loc="lower right")
axs[1].set_title("Accuracy_A")
axs[1].sharex(axs[0])

for ax in axs.flat:
    ax.set(xlabel='Epoch')
fig.show()

In [ ]:
# Graphs
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].plot(history_B.history['loss'], label="Training Loss") 
axs[0].plot(history_B.history['val_loss'], label="Validation Loss")
axs[0].set_title("Loss_B")
axs[0].legend(loc="upper right")
axs[1].plot(history_B.history['accuracy'], label="Trainig Accuracy") 
axs[1].plot(history_B.history['val_accuracy'], label="Validation Accuracy")
axs[1].legend(loc="lower right")
axs[1].set_title("Accuracy_B")
axs[1].sharex(axs[0])

for ax in axs.flat:
    ax.set(xlabel='Epoch')
fig.show()

In [ ]:
# Graphs
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].plot(history_AB.history['loss'], label="Training Loss") 
axs[0].plot(history_AB.history['val_loss'], label="Validation Loss")
axs[0].set_title("Loss_AB")
axs[0].legend(loc="upper right")
axs[1].plot(history_AB.history['accuracy'], label="Trainig Accuracy") 
axs[1].plot(history_AB.history['val_accuracy'], label="Validation Accuracy")
axs[1].legend(loc="lower right")
axs[1].set_title("Accuracy_AB")
axs[1].sharex(axs[0])

for ax in axs.flat:
    ax.set(xlabel='Epoch')
fig.show()